# CUDA编程模型---初识CUDA

### 本次实验将介绍如何：
1. 编写第一个Cuda程序
2. 利用NVCC进行编译
3. 编写Makefile文件
4. 线程索引
5. 利用nvprof查看程序性能

----
## 1.编写第一个Cuda程序
- 关键词："\_\_global\_\_" ,  <<<...>>>  ,  .cu

在当前的目录下创建一个名为hello_cuda.cu的文件，编写第一个Cuda程序：
- 当我们编写一个hello_word程序的时候，我们通常会这样写：
```c
    #include <stdio.h>

    void hello_from_cpu()
    {
        printf("Hello World from the CPU!\n");
    }

    int main(void)
    {
        hello_from_cpu();
        return 0;
    }
```

- 如果我们要把它改成调用GPU的时候，我们需要在void hello_from_cpu()之前加入 \_\_global\_\_标识符，并且在调用这个函数的时候添加<<<...>>>来设定你需要多少个线程来执行这个函数
- 在当前的目录下创建一个名为[hello_cuda.cu](hello_cuda.cu)的文件，更改上述程序，将它改为在GPU上执行的程序，如果遇到麻烦，请参考[result_1.cu](result_1.cu)

----
## 2.编写完成之后，我们要开始编译并执行程序，在这里我们可以利用nvcc进行编译，指令如下：

In [23]:
!/usr/local/cuda/bin/nvcc hello_cuda.cu -o hello_cuda -run

Hello World from the GPU!


----
## 3.这里我们也可以利用编写Makefile的方式来进行编译，一个简单的例子可以参考[Makefile](Makefile)

In [24]:
!make

make: 'hello_cuda' is up to date.


然后我们就可以得到一个名为hello_cuda的程序，我们开始执行一下

In [25]:
!./hello_cuda

Hello World from the GPU!


In [42]:
# clean output file
!make clean

rm -rf ./hello_cuda
rm -rf *.o


接下来我们尝试多个文件协同编译, 修改[Makefile](Makefile)文件:
1. 编译hello_from_gpu.cu文件生成hello_from_gpu.o
2. 编译hello_cuda_01.cu和上一步生成的hello_from_gpu.o, 生成./hello_cuda_multi_file

如果遇到麻烦, 请参考[Makefile_Multi_file](Makefile_Multi_file)

In [30]:
#此处通过-f来指定您使用的编译文件
!make -f Makefile_Multi_file

/usr/local/cuda/bin/nvcc --device-c hello_from_gpu.cu -o hello_from_gpu.o
/usr/local/cuda/bin/nvcc  hello_cuda_01.cu hello_from_gpu.o -o ./hello_cuda_multi_file


In [31]:
!./hello_cuda_multi_file

Hello World from the GPU!


In [41]:
# clean output file
!make -f Makefile_Multi_file clean

rm -rf ./hello_cuda_multi_file
rm -rf *.o


这时，您已经完成了第一个Cuda程序，接下来修改<<<...>>>里面的信息，查看显示效果，如果遇到麻烦，请参考[hello_cuda_02.cu](hello_cuda_02.cu)

----
## 4.线程索引
当我们在讨论GPU和CUDA时，我们一定会考虑如何调用每一个线程，如何定为每一个线程。其实，在CUDA编程模型中，每一个线程都有一个唯一的标识符或者序号，而我们可以通过threadIdx来得到当前的线程在线程块中的序号,通过blockIdx来得到该线程所在的线程块在grid当中的序号，即：

* **threadIdx.x** 是执行当前kernel函数的线程在block中的x方向的序号  

* **blockIdx.x** 是执行当前kernel函数的线程所在block，在grid中的x方向的序号

* **blockDim.x** 是执行当前kernel函数的线程所在的block在x方向包含多少个线程

* **gridDim.x** 是执行当前kernel函数的grid在x方向包含多少个block


接下来创建[Index_of_thread.cu](Index_of_thread.cu)文件，并在核函数中打印执行该核函数的线程编号和所在的线程块的编号，如果遇到麻烦，请参考[result_2.cu](result_2.cu)

创建好了之后，我们开始编译

In [44]:
# <<<2, 4>>>
!/usr/local/cuda/bin/nvcc Index_of_thread.cu -o Index_of_thread

执行Index_of_thread

In [45]:
!./Index_of_thread

Hello World from the GPU! blockIdx is 0, threadIdx is 0!
Hello World from the GPU! blockIdx is 0, threadIdx is 1!
Hello World from the GPU! blockIdx is 0, threadIdx is 2!
Hello World from the GPU! blockIdx is 0, threadIdx is 3!
Hello World from the GPU! blockIdx is 1, threadIdx is 0!
Hello World from the GPU! blockIdx is 1, threadIdx is 1!
Hello World from the GPU! blockIdx is 1, threadIdx is 2!
Hello World from the GPU! blockIdx is 1, threadIdx is 3!


修改`<<<...>>>`中的值，查看执行结果，这里建议分三组：`<<<33,5>>>`, `<<<5,33>>>`,`<<<5,65>>>`, 然后重新编译并执行， 前面是block num, 后面是 thread num in each block

In [52]:
# <<<5, 2>>>
!/usr/local/cuda/bin/nvcc Index_of_thread.cu -o Index_of_thread

In [53]:
!./Index_of_thread

Hello World from the GPU! blockIdx is 0, threadIdx is 0!
Hello World from the GPU! blockIdx is 0, threadIdx is 1!
Hello World from the GPU! blockIdx is 0, threadIdx is 2!
Hello World from the GPU! blockIdx is 0, threadIdx is 3!
Hello World from the GPU! blockIdx is 1, threadIdx is 0!
Hello World from the GPU! blockIdx is 1, threadIdx is 1!
Hello World from the GPU! blockIdx is 1, threadIdx is 2!
Hello World from the GPU! blockIdx is 1, threadIdx is 3!


----
## 5.利用nvprof进行查看程序性能

In [54]:
!sudo /usr/local/cuda/bin/nvprof  ./Index_of_thread

==22412== NVPROF is profiling process 22412, command: ./Index_of_thread
==22412== Warning: Unified Memory Profiling is not supported on the underlying platform. System requirements for unified memory can be found at: http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#um-requirements
Hello World from the GPU! blockIdx is 0, threadIdx is 0!
Hello World from the GPU! blockIdx is 0, threadIdx is 1!
Hello World from the GPU! blockIdx is 0, threadIdx is 2!
Hello World from the GPU! blockIdx is 0, threadIdx is 3!
Hello World from the GPU! blockIdx is 1, threadIdx is 0!
Hello World from the GPU! blockIdx is 1, threadIdx is 1!
Hello World from the GPU! blockIdx is 1, threadIdx is 2!
Hello World from the GPU! blockIdx is 1, threadIdx is 3!
==22412== Profiling application: ./Index_of_thread
==22412== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  95.245us         1  95.245us  95.245us  95.245us  hello_fro

- Profiling result：是GPU（kernel函数）上运行的时间
- API calls：是在cpu上测量的程序调用API的时间

In [55]:
# clean output file
!rm -f ./Index_of_thread

课后作业：
1. 利用Makefile规则，尝试编写批量编译工具，比如：同时编译5个cuda程序。
2. 利用Makefile规则，尝试加入链接库，比如：加入cuBLAS库编译cuda程序。
3. 阅读Cuda sample code，尝试编写程序得到当前GPU的属性参数等。
4. 阅读[nvprof](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#nvprof-overview) 说明文档，了解更多nvprof的使用方法，为后续课程中使用做